Evaluate experimental design using D-Efficiency.

**Definitions**:

$\mathbf{X}$ is the model matrix: A row for each run and a column for each term in the model.

For instance, a model assuming only main effects:

$\mathbf{Y} = \mathbf{X} \beta + \alpha$

$\mathbf{X}$ will contain $p = m + 1$ columns (number of factors + intercept).


D-optimality:


*D-efficiency* $=  \left(\frac{1}{n}|\mathbf{X}'\mathbf{X}|^{1/p}\right)$

**D-efficiency**

D-efficiency compares the design $\mathbf{X}$ with the D-optmial design $\mathbf{X_D}$ for the assumed model: 

*D-efficiency* $= \left[ \frac{|\mathbf{X}'\mathbf{X}|}{|\mathbf{X_D}'\mathbf{X_D}|} \right]^{1/p}$


In JMP, the D-efficiency compares the design with an orthogonal design in terms of D-optimality criterion:

*D-efficiency* $= 100 \left(\frac{1}{n}|\mathbf{X}'\mathbf{X}|^{1/p}\right)$

**Designs**

D-optimal designs maximize $D$:

$D  = |\mathbf{X}'\mathbf{X}|$

(no need for the other terms because they are constant)

D-optimal split-plot designs maximize:

$D  = |\mathbf{X}'\mathbf{V}^{-1}\mathbf{X}|$

where $\mathbf{V}^{-1}$ is the block diagonal covariance matrix of the responses.

Split-plot designs are those that some factors are harder to vary than others. The covariance indiciates the ratio between the whole variance and the subplot variance to the error variance. 

**Estimation efficiency**

There are several parameters (see JMP guide), but they are related. The basic is the relateive std error to estimate, i.e., how large the standard errors of the model's parameter estimates are relative to the error standard deviation.

*SE* $= \sqrt{\left(\mathbf{X}'\mathbf{X}\right)_{ii}^{-1}}$

where $\left(\mathbf{X}'\mathbf{X}\right)_{ii}^{-1}$ is the $i$ diagonal of $\left(\mathbf{X}'\mathbf{X}\right)^{-1}$.

# Example

An example for 2-factor model with main effects:

In [1]:
import numpy as np

In [2]:
X1 = np.matrix( "[1 1 1; 1 -1 1; 1 -1 -1]")
X1

In [320]:
def Deff(X):
    # D-efficiency
    return (100.0/X.shape[0]) * ( np.linalg.det( np.dot( np.transpose( X ), X ) )**(1.0/X.shape[1]))
def Dopt(X):
    # D-optimality
    return np.linalg.det( np.dot( np.transpose( X ), X ) )
def SE(X):
    # Estimation efficiency
    return np.diag( np.linalg.inv( np.dot( np.transpose( X ), X ) ) )
def Contrib(X):
    cn = []
    for i in range(0, X.shape[0]):
        cn.append( Dopt( np.vstack( [X[:i,:], X[(i+1):,:]]  )  ) )
    return cn
def VarAdd(X,xj):
    # Variance of adding/removing one
    return np.dot( np.dot( np.transpose(xj) , np.linalg.inv( np.dot( np.transpose( X ), X) ) ), xj )
    

In [163]:
Dopt( np.vstack( [X[:i,:], X[i:,:]]  )   )

311.9999999999998

In [164]:
Contrib(X1)

(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)
(10, 6) (10, 6)


[311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998,
 311.9999999999998]

In [13]:
SE(X1)

array([0.5, 0.5, 0.5])

# Algorithm

In [307]:
import itertools


p = 10 # Number of factors  print(w,Deff(X),i,j
n = 24 # Number of runs
# Initial design
X = np.random.randint(0,2,(n,p+1))
val = []
for x in np.arange(p):
    val.append([0,1])
ffact = []
for m in itertools.product(*val):
    ffact.append(m)
ffact = np.array(ffact)
X = ffact[np.random.randint(0,len(ffact),n),:]
J = 0
w = 0
while ( (J < 1e4) and (w < 10000) ):
    d2 = None
    d6 = None
    w += 1
    try:
        d1 = Deff( X )
        d2 = Dopt( X )
        d3 = SE( X )
        d4 = Contrib( X )
    except:
        continue
    J = max(J, d1)
    i = np.argmin( d4 )
    j = np.argmax( d3 )
    X1 = X.copy()
    k = 0
    while( k < 10 ):
        i = np.argsort( d4 )[ np.random.randint(0,5)]
        j = np.flip( np.argsort( d3 ) )[0] # [ np.random.randint(0,5)]
        X1[i,:] = ffact[np.random.randint(0,len(ffact),1), :]
#        if X[i,j] == 0:
#            X1[i,j] = 1
#        else:
#            X1[i,j] = 0
        k += 1
    try:
        d5 = Deff( X1 )
        d6 = Dopt( X1 )
        d7 = SE( X1 )
        d8 = Contrib( X1 )
  
    except:
        continue
    if d6 > d2:
        X = X1
        print(w,J,d1,d2,d6,i,j)

print(w,J,d1,d2,d6,i,j)
    

39 24.757137280437647 24.757137280437647 54842426.99999994 55795928.00000006 16 6
70 24.799847417156084 24.799847417156084 55795928.00000006 63224372.00000023 2 2
270 25.111763392576464 25.111763392576464 63224372.00000023 64715250.00000028 12 1
1070 25.170359681548877 25.170359681548877 64715250.00000028 78980367.0000001 5 1
2767 25.676786679666098 25.676786679666098 78980367.0000001 87557751.00000007 12 0
3639 25.942881867260958 25.942881867260958 87557751.00000007 90560945.00000003 2 7
10000 26.030520536933427 26.030520536933427 90560945.00000003 6691743.000000008 20 0


In [435]:
p = 100 # Number of factors
n = 150 # Number of runs
m = 100 # Sampled design space per iteration

# Here we generate a full factorial but this is not possible for large designs
# Replace by random sampling, ideally some descent algorithm (TBD)
# For categorical variables, we randomize the levels that are then mapped into the dummy variables
FULL = False
if FULL:
    val = []
    for x in np.arange(p+1):
        val.append([-1,1])
    ffact = []
    for m in itertools.product(*val):
        ffact.append(m)
    ffact = np.array(ffact)
# Initial design: could we do something better than a purely random start?
X = np.array([-1,1])[ np.random.randint(0,2,(n,p+1)) ]
# D-Efficiency of the initial design
# Here I have implemented a simple DETMAX algorithm. At each iteration: 
#  - remove the design with the lowest variance
#  - add the design with the highest variance
# Many more efficent variants exist (kl-exchange, etc..)
J = Deff(X)
print(J)
w = 0
while ((J<99.0) and (w < 100)):
    # X1 is the design space sample in the iteration. 
    # Here we loop through the full factorial, which is computationally expensive
    # First thing to fdo is to change it to random generation of a subset of the full library
    # It would be better to move across some surface like gradient descent...
    if FULL:
        X1 = ffact
    else:
        X1 = np.array([-1,1])[ np.random.randint(0,2,(m,p+1)) ]
    sub = []
    for i in np.arange(X.shape[0]):
        sub.append( VarAdd(X, X[i,:]) )
    w += 1
    Xsub = None
    dList = np.argsort( sub )[0:1]
    for i in np.arange(X.shape[0]):
        if i in dList:
            continue
        else:
            if Xsub is None:
                Xsub = X[i,:]
            else:
                Xsub = np.vstack( [Xsub, X[i,:]] )
    add = []
    for j in np.arange(X1.shape[0]):
        add.append( VarAdd( Xsub, X1[j,:] ) ) 
    aList = np.flip( np.argsort( add ) )[0:1]
    Xn = Xsub
    for j in aList:
        Xn = np.vstack( [Xn, X1[j,:] ] )
    if w % 100 == 0:
        print(w,J,i,j, Dopt(X), Dopt(Xsub), Dopt(Xn))
    if Dopt(Xn) > Dopt(X):
        X = Xn
        J = Deff(X)
    elif Dopt(Xn) == Dopt(X):
        break
print(w,J,i,j)
    

63.10813627200491
100 75.63574184274607 149 90 3.440735760266953e+207 1.331020674558307e+207 3.5931443869773624e+207
100 75.66820654669202 149 90


In [437]:
X.shape

(150, 101)

In [255]:
X = np.random.randint(0,2,(n,p+1))
X1 = np.random.randint(0,2,(n,p+1))
add = []
for i in np.arange(X1.shape[0]):
    add.append( VarAdd(X, X1[i,:]) )
sub = []
for i in np.arange(X.shape[0]):
    sub.append( VarAdd(X, X[i,:]) )

In [256]:
sub

[0.22018483624943913,
 0.12499237326155221,
 0.25479048900852397,
 0.25334768954688197,
 0.2410013458950202,
 0.26289816061013904,
 0.32101929116195604,
 0.33746074472857784,
 0.37110812023328843,
 0.18941229250785102,
 0.30099237326155226,
 0.2430650515926424,
 0.23945805293853742,
 0.2136958277254374,
 0.26101390758187526,
 0.20359264244055628,
 0.21644145356662178,
 0.3014481830417228,
 0.2090121130551817,
 0.2136958277254374,
 0.37379991027366527,
 0.1528183041722746,
 0.25568775235531627,
 0.23906325706594883]

In [211]:
Dopt( X1 )

83072.00000000001

In [171]:
d4

[32.0,
 144.00000000000014,
 110.00000000000004,
 97.99999999999986,
 140.0000000000002,
 127.99999999999997,
 116.00000000000009,
 110.00000000000004,
 127.99999999999997,
 98.00000000000013]

In [56]:
 np.dot( np.transpose( X), np.array( X ) ) 

array([[2, 0, 1, 2],
       [0, 0, 0, 0],
       [1, 0, 1, 1],
       [2, 0, 1, 2]])

In [36]:
np.transpose( X )

array([[1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1],
       [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
        0, 1],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 0],
       [1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 1],
       [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 0],
       [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 1],
       [1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0]])

In [372]:
np.linalg.det( np.dot( np.transpose( X ), X  ) )

671579.9999999993

In [373]:
24**11

1521681143169024

In [403]:
import dexpy.optimal
from dexpy.model import ModelOrder
reaction_design = dexpy.optimal.build_optimal(50, run_count=64, order=ModelOrder.linear)
#reaction_design

In [402]:
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

dexpy library works fine, but it does not work with dummy variables. Therefore, if we add dummy variables, there will be clashes between levels in the factor (multiple level set simultaneously). Also, if we increase the order of the model, it will generate intermediate values that are not actually meaningful for dummy variables. 

Dummy variables could probably be better processed by generating a full factorial with labels and then convert to dummy after sampling.

In [406]:
reaction_design

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,...,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0
3,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,...,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
4,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0
5,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,...,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0
6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,...,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,...,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
8,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0
9,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,...,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0


In [408]:
np.matrix( "[1 1 1; 2 3 4]")

matrix([[1, 1, 1],
        [2, 3, 4]])

In [409]:
import pandas as pd

In [413]:
df = pd.read_excel('/mnt/SBC1/data/OptimalDesign/data/CD.xlsx')
XX = np.array( df.iloc[:,0:5] )

In [414]:
Deff(XX)

100.00000000000004

In [416]:
X = np.array([-1,-1])[ np.random.randint(0,2,(n,p+1)) ]

In [417]:
X

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])